In [1]:
import sys
import polars as pl
import boto3
import time
from IPython.display import display
import datetime
import math
from pathlib import PurePosixPath
print(f"{sys.version=}")

sys.version='3.14.0 (main, Oct  7 2025, 09:34:52) [Clang 17.0.0 (clang-1700.3.19.1)]'


# Список silver S3

In [2]:
s3 = boto3.resource(
    's3',
    endpoint_url='http://localhost:9000',
    aws_access_key_id='minioadmin',
    aws_secret_access_key='minioadmin',
    aws_session_token=None,
    config=boto3.session.Config(signature_version='s3v4'),
    verify=False
)
storage_options = {
    "aws_endpoint_url": "http://localhost:9000", # Важно: aws_endpoint_url (иногда endpoint_url)
    "aws_access_key_id": "minioadmin",
    "aws_secret_access_key": "minioadmin",
    "aws_region": "us-east-1", # Для MinIO часто можно оставить us-east-1
    "aws_allow_http": "true",  # Разрешить HTTP (без SSL)
}

In [3]:
bucket = s3.Bucket('silver')

In [8]:
def silver_path2dt(obj):
    return datetime.datetime(**{item.split("=",1)[0]: int(item.split("=",1)[1]) for item in PurePosixPath(obj.key).parts if "=" in item})
objects = sorted(bucket.objects.all(), key=silver_path2dt)

In [9]:
objects[0], objects[1]

(s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet'),
 s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet'))

In [32]:
total_size_mb = 0
for obj in objects:
    key = obj.key
    size_mb = obj.size//1e+6
    total_size_mb += size_mb
    print(key, size_mb, obj.last_modified)

hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet 2.0 2025-12-12 09:04:06.609000+00:00
hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet 4.0 2025-12-12 09:04:38.370000+00:00
hh/vacancies/year=2025/month=11/day=14/part-c6077654-fe00-4dbe-8e13-9bc4e0f56fa8.parquet 4.0 2025-12-12 09:05:07.196000+00:00
hh/vacancies/year=2025/month=11/day=15/part-5157769f-7a83-429a-bb82-1bdef8d967f7.parquet 1.0 2025-12-12 09:05:16.560000+00:00
hh/vacancies/year=2025/month=11/day=16/part-dd809784-9f1d-49b5-a2f3-65d8b023a6b2.parquet 0.0 2025-12-12 09:05:23.998000+00:00
hh/vacancies/year=2025/month=11/day=17/part-aaa5d138-e5fa-4d3b-8a81-151e2e7e774b.parquet 7.0 2025-12-12 09:06:17.416000+00:00
hh/vacancies/year=2025/month=11/day=18/part-7e1d1cdf-0933-4c15-9ee3-c2b62f0d5681.parquet 6.0 2025-12-12 09:07:02.054000+00:00
hh/vacancies/year=2025/month=11/day=19/part-1471860e-073d-4cdc-bb43-7ed3639cb269.parquet 6.0 2025-12-12 09:07:42.21600

In [34]:
print('Silver layer size', total_size_mb, 'mb')

Silver layer size 146.0 mb


In [11]:
# objects = [obj for obj in objects if silver_path2dt(obj).day >8 and silver_path2dt(obj).month == 12]
# objects

# Общее кол-во вакансий в S3

In [12]:
for obj in objects:
    schema = pl.scan_parquet(f"s3://silver/{obj.key}",storage_options=storage_options).collect_schema()
    print(obj, len(schema))

s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=12/part-bba9b6b6-a3f5-4165-bbd9-b34ab9a7fbe6.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=13/part-e374b7d5-01a4-49c0-b515-84e86c8283cd.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=14/part-c6077654-fe00-4dbe-8e13-9bc4e0f56fa8.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=15/part-5157769f-7a83-429a-bb82-1bdef8d967f7.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=16/part-dd809784-9f1d-49b5-a2f3-65d8b023a6b2.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=17/part-aaa5d138-e5fa-4d3b-8a81-151e2e7e774b.parquet') 61
s3.ObjectSummary(bucket_name='silver', key='hh/vacancies/year=2025/month=11/day=18/part-7e1d1cdf-0933-4c15-9ee3-c2b62f0d5681.parquet') 61
s3.ObjectSummary(bucket_name='silv

In [13]:
df_lazy = pl.scan_parquet(
    [f"s3://silver/{obj.key}" for obj in objects],
    storage_options=storage_options,
)

In [14]:
df_lazy.select(
    pl.col('id').count().alias('id_cnt'),
    pl.col('id').unique().count().alias('unique_id_cnt')
).collect()

id_cnt,unique_id_cnt
u32,u32
689224,689224


In [25]:
df_lazy.tail(1).collect().glimpse()

Rows: 1
Columns: 61
$ id                                    <str> '128392992'
$ name                                  <str> 'Бетонщик (Забайкальский край)'
$ description                           <str> null
$ branded_description                   <str> null
$ key_skills                <list[struct[1]]> null
$ schedule                        <struct[2]> {'id': 'flyInFlyOut', 'name': 'Вахтовый метод'}
$ accept_handicapped                   <bool> null
$ accept_kids                          <bool> null
$ experience                      <struct[2]> {'id': 'between1And3', 'name': 'От 1 года до 3 лет'}
$ address                        <struct[10]> {'city': 'посёлок городского типа Новая Чара', 'street': None, 'building': None, 'lat': 56.79906, 'lng': 118.27571, 'description': None, 'raw': 'посёлок городского типа Новая Чара', 'metro': None, 'metro_stations': [], 'id': '19428704'}
$ employer                        <struct[9]> {'id': '11875449', 'name': 'ПрессБук', 'url': 'https://api.hh.ru/em

# Lil EDA

In [43]:
df_lazy = (
    pl.scan_parquet([f"s3://silver/{obj.key}" for obj in objects], storage_options=storage_options)

    # Published Date transfromation from UTC to Local
    .with_columns(
        published_at_local = (
                pl.col('published_at_utc').dt.replace_time_zone(None) +
                pl.duration(minutes=pl.col('published_at_offset')
            )
        )
    )
    .with_columns(
        published_date=pl.col('published_at_local').dt.date()
    )
)

## Describe

In [53]:
df_lazy_described = df_lazy.describe()

In [77]:
from IPython.display import display, HTML

display(HTML("""
<style>
/* classic notebook */
div.output_area pre {
  white-space: pre !important;   /* не переносить строки */
}

/* jupyterlab */
div.jp-OutputArea-output pre {
  white-space: pre !important;   /* не переносить строки */
}
</style>
"""))

with pl.Config(tbl_cols=-1, tbl_formatting='MARKDOWN', tbl_width_chars=-1):
    print(df_lazy_described)

shape: (9, 64)
| statistic  | id        | name                            | description | branded_description | key_skills | schedule | accept_handicapped | accept_kids | experience | address  | employer | salary   | archived | response_url                    | specializations | professional_roles | driver_license_types | contacts | response_letter_required | type     | has_test | test     | alternate_url                   | working_days | working_time_intervals | working_time_modes | accept_temporary | languages | immediate_redirect_url          | video_vacancy | brand_snippet | is_adv_vacancy | internship | department | show_contacts | apply_alternate_url             | work_format | relations | accept_incomplete_resumes | premium  | employment | employment_form | adv_context | sort_point_distance | branding | insider_interview | fly_in_fly_out_duration | url                             | salary_range | work_schedule_by_days | show_logo_in_search | area     | snippet  | night_shifts |

## Check nulls

In [87]:
df_lazy.null_count().collect().glimpse()

Rows: 1
Columns: 63
$ id                        <u32> 0
$ name                      <u32> 0
$ description               <u32> 689224
$ branded_description       <u32> 689224
$ key_skills                <u32> 689224
$ schedule                  <u32> 0
$ accept_handicapped        <u32> 689224
$ accept_kids               <u32> 689224
$ experience                <u32> 0
$ address                   <u32> 118621
$ employer                  <u32> 0
$ salary                    <u32> 117417
$ archived                  <u32> 0
$ response_url              <u32> 681597
$ specializations           <u32> 689224
$ professional_roles        <u32> 0
$ driver_license_types      <u32> 689224
$ contacts                  <u32> 158673
$ response_letter_required  <u32> 0
$ type                      <u32> 0
$ has_test                  <u32> 0
$ test                      <u32> 689224
$ alternate_url             <u32> 0
$ working_days              <u32> 0
$ working_time_intervals    <u32> 0
$ working_time_modes

## Слишком много null'ов в silver, проверка заполненности данных в bronze

In [93]:
bucket = s3.Bucket('bronze')

In [113]:
report_dt = "2025-12-11"
report_dt_objects = [o for o in bucket.objects.filter(Prefix=f"hh/vacancies/date={report_dt}")]
len(report_dt_objects)

29

In [124]:
for obj in report_dt_objects:
    s3path = f"s3://bronze/{obj.key}"
    lazy_bronze = pl.scan_ndjson(s3path, storage_options=storage_options, infer_schema_length=None)
    null_cnt = lazy_bronze.null_count().collect()
    print(null_cnt)
    

shape: (1, 49)
┌─────┬─────┬─────┬─────┬─────┬─────┬─────┬─────┬───┬─────┬─────┬─────┬─────┬─────┬────────┬───────┐
│ int ┆ dep ┆ typ ┆ adv ┆ sal ┆ are ┆ is_ ┆ emp ┆ … ┆ pro ┆ exp ┆ emp ┆ sho ┆ pub ┆ immedi ┆ brand │
│ ern ┆ art ┆ e   ┆ _re ┆ ary ┆ a   ┆ adv ┆ loy ┆   ┆ fes ┆ eri ┆ loy ┆ w_c ┆ lis ┆ ate_re ┆ _snip │
│ shi ┆ men ┆ --- ┆ spo ┆ _ra ┆ --- ┆ _va ┆ er  ┆   ┆ sio ┆ enc ┆ men ┆ ont ┆ hed ┆ direct ┆ pet   │
│ p   ┆ t   ┆ u32 ┆ nse ┆ nge ┆ u32 ┆ can ┆ --- ┆   ┆ nal ┆ e   ┆ t   ┆ act ┆ _at ┆ _url   ┆ ---   │
│ --- ┆ --- ┆     ┆ _ur ┆ --- ┆     ┆ cy  ┆ u32 ┆   ┆ _ro ┆ --- ┆ --- ┆ s   ┆ --- ┆ ---    ┆ u32   │
│ u32 ┆ u32 ┆     ┆ l   ┆ u32 ┆     ┆ --- ┆     ┆   ┆ les ┆ u32 ┆ u32 ┆ --- ┆ u32 ┆ u32    ┆       │
│     ┆     ┆     ┆ --- ┆     ┆     ┆ u32 ┆     ┆   ┆ --- ┆     ┆     ┆ u32 ┆     ┆        ┆       │
│     ┆     ┆     ┆ u32 ┆     ┆     ┆     ┆     ┆   ┆ u32 ┆     ┆     ┆     ┆     ┆        ┆       │
╞═════╪═════╪═════╪═════╪═════╪═════╪═════╪═════╪═══╪═════╪═════╪═════╪═════

In [143]:
obj = report_dt_objects[4]
s3path = f"s3://bronze/{obj.key}"
lazy_bronze = pl.scan_ndjson(s3path, storage_options=storage_options, infer_schema_length=None)

In [144]:
'description' in lazy_bronze.collect_schema().names()

False

In [145]:
silver_schema = set(df_lazy.collect_schema().names())
bronze_schema = set(lazy_bronze.collect_schema().names())
len(silver_schema), len(bronze_schema)

(63, 49)

In [146]:
len(silver_schema & bronze_schema)

47

In [147]:
len(silver_schema - bronze_schema)

16

In [148]:
silver_schema - bronze_schema

{'accept_handicapped',
 'accept_kids',
 'branded_description',
 'created_at_offset',
 'created_at_utc',
 'date',
 'description',
 'driver_license_types',
 'key_skills',
 'languages',
 'published_at_local',
 'published_at_offset',
 'published_at_utc',
 'published_date',
 'specializations',
 'test'}

In [149]:
'test' in silver_schema

True

## professional_role count

In [84]:
df_lazy.head(1).collect().glimpse()

Rows: 1
Columns: 63
$ id                                    <str> '107445022'
$ name                                  <str> 'Кладовщик'
$ description                           <str> null
$ branded_description                   <str> null
$ key_skills                <list[struct[1]]> null
$ schedule                        <struct[2]> {'id': 'shift', 'name': 'Сменный график'}
$ accept_handicapped                   <bool> null
$ accept_kids                          <bool> null
$ experience                      <struct[2]> {'id': 'noExperience', 'name': 'Нет опыта'}
$ address                        <struct[10]> null
$ employer                        <struct[9]> {'id': '1942330', 'name': 'Пятёрочка', 'url': 'https://api.hh.ru/employers/1942330', 'alternate_url': 'https://hh.ru/employer/1942330', 'logo_urls': {'original': 'https://img.hhcdn.ru/employer-logo-original-round/7101933.png', '90': 'https://img.hhcdn.ru/employer-logo-round/7101934.png', '240': 'https://img.hhcdn.ru/employer-logo-ro

In [92]:
q = (df_lazy.with_columns(
        pl.col('professional_roles').list.get(0).struct.field('name').alias('professional_role')
    )
)
q.select('professional_role').unique().count().collect()

professional_role
u32
174
